In [1]:
import numpy as np
import cv2
import pandas as pd
import tensorflow
import os
import matplotlib





Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


In [2]:
def importDataInfo():
    columns=['centre','left','right','steering','throttle','brake','speed']
    data=pd.read_csv("driving_log.csv",names=columns)
    return data

In [3]:


data=importDataInfo()

In [4]:
data['centre'][0]

'C:\\Users\\umer\\Documents\\game data\\IMG\\center_2024_03_18_23_14_14_648.jpg'

In [5]:
data.steering

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
6041    0.0
6042    0.0
6043    0.0
6044    0.0
6045    0.0
Name: steering, Length: 6046, dtype: float64

In [6]:
# a=cv2.imread(data['centre'][0])
# print(a)
# cv2.imshow("win",a)
# cv2.waitKey(0)

In [7]:
data.centre[0]

'C:\\Users\\umer\\Documents\\game data\\IMG\\center_2024_03_18_23_14_14_648.jpg'

In [8]:
data.steering.shape

(6046,)

In [9]:
data.shape

(6046, 7)

In [19]:
train_images=[]
train_labels=[]
for i in range(6046):
    a=cv2.imread(data.centre[i])
    a=cv2.resize(a,(200,66))
    train_images.append(a)
    train_labels.append(data.steering[i])


In [20]:
len(train_images)

6046

In [21]:
train_images=np.array(train_images)
train_labels=np.array(train_labels)

In [22]:
train_labels.shape


(6046,)

In [23]:
train_images.shape

(6046, 66, 200, 3)

In [24]:
train_images[1].shape

(66, 200, 3)

error: OpenCV(4.9.0) :-1: error: (-5:Bad argument) in function 'resize'
> Overload resolution failed:
>  - Can't parse 'dsize'. Expected sequence length 2, got 3
>  - Can't parse 'dsize'. Expected sequence length 2, got 3


In [25]:
np.histogram(data["steering"])

(array([   9,   37,   76,  328,  474, 4843,  185,   44,   30,   20],
       dtype=int64),
 array([-1. , -0.8, -0.6, -0.4, -0.2,  0. ,  0.2,  0.4,  0.6,  0.8,  1. ]))

In [26]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,Flatten,Dense

In [27]:
model=Sequential()
model.add(Convolution2D(24,(5,5),(2,2),input_shape=(66,200,3),activation='elu'))
model.add(Convolution2D(36,(5,5),(2,2),activation='elu'))
model.add(Convolution2D(48,(5,5),(2,2),activation='elu'))
model.add(Convolution2D(64,(3,3),activation='elu'))
model.add(Convolution2D(64,(3,3),activation='elu'))

model.add(Flatten())
model.add (Dense(100,activation='elu'))
model.add (Dense(50,activation='elu'))
model.add (Dense(10,activation='elu'))
model.add (Dense(1))

optimizer = tensorflow.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='mse', metrics=['accuracy'])


In [29]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 31, 98, 24)        1824      
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 47, 36)        21636     
                                                                 
 conv2d_2 (Conv2D)           (None, 5, 22, 48)         43248     
                                                                 
 conv2d_3 (Conv2D)           (None, 3, 20, 64)         27712     
                                                                 
 conv2d_4 (Conv2D)           (None, 1, 18, 64)         36928     
                                                                 
 flatten (Flatten)           (None, 1152)              0         
                                                                 
 dense (Dense)               (None, 100)               1

In [31]:
model.fit(train_images,train_labels,batch_size=200,epochs=10,validation_split=0.1)

Epoch 1/10
28/28 [==============================] - 32s 1s/step - loss: 0.1159 - accuracy: 0.6883 - val_loss: 0.1227 - val_accuracy: 0.7091
Epoch 2/10
28/28 [==============================] - 29s 1s/step - loss: 0.0953 - accuracy: 0.6995 - val_loss: 0.1061 - val_accuracy: 0.7289
Epoch 3/10
28/28 [==============================] - 28s 1s/step - loss: 0.0813 - accuracy: 0.7045 - val_loss: 0.0928 - val_accuracy: 0.7488
Epoch 4/10
28/28 [==============================] - 34s 1s/step - loss: 0.0706 - accuracy: 0.7100 - val_loss: 0.0825 - val_accuracy: 0.7587
Epoch 5/10
28/28 [==============================] - 43s 2s/step - loss: 0.0620 - accuracy: 0.7146 - val_loss: 0.0746 - val_accuracy: 0.7702
Epoch 6/10
28/28 [==============================] - 41s 1s/step - loss: 0.0549 - accuracy: 0.7173 - val_loss: 0.0687 - val_accuracy: 0.7686
Epoch 7/10
28/28 [==============================] - 56s 2s/step - loss: 0.0497 - accuracy: 0.7203 - val_loss: 0.0635 - val_accuracy: 0.7702
Epoch 8/10
28/28 [==

In [32]:
model.save('model.h5')

c:\Users\umer\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3078: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [13]:
from tensorflow.keras.models import load_model
model=load_model('model.h5')

In [14]:
import eventlet
from flask import Flask
import socketio
from io import BytesIO
import base64

In [15]:
def preeprocess(img):
    img=img[60:135,:,:]
    img=cv2.cvtColor(img,cv2.COLOR_RGB2YUV)
    img=cv2.GaussianBlur(img,(3,3),0)
    img=cv2.resize(img,(200,66))
    img=img/255
    return img

In [16]:
sio = socketio.Server()
app=Flask(__name__)
maxspeed=10
@sio.on('telemetry')
def telemetry(sid,data):
    speed=float(data['speed'])
    image=Image.open(BytesIO(base64.b64decode(data['image'])))
    image=np.asarray(image)
    image=preeprocess(image)
    image=np.array([image])
    steering=float(model.predict(image))
    throttle=1-speed/maxspeed
    print('{}{}{}'.format(steering,throttle,speed))
    sendcontrol(steering,throttle)

In [17]:
@sio.on('connect')
def connect(sid,environ):
    print('connected')
    sendcontrol(0,0)

In [8]:
def sendcontrol(steering,throttle):
    sio.emit('steer' ,data={
        'steering_angle':steering.__str__(),
        'throttle':throttle.__str__()
    }
    )

In [12]:
app=socketio.Middleware(sio,app)
eventlet.wsgi.server(eventlet.listen(('',4567) ),app)

(13420) wsgi starting up on http://0.0.0.0:4567
(13420) accepted ('127.0.0.1', 55296)
wsgi exiting
127.0.0.1 - - [19/Mar/2024 04:27:59] "GET /socket.io/?EIO=4&transport=websocket HTTP/1.1" 200 0 80.044063
(13420) wsgi exited, is_accepting=True


In [1]:
print('Setting UP')
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import socketio
import eventlet
import numpy as np
from flask import Flask
from tensorflow.keras.models import load_model
import base64
from io import BytesIO
from PIL import Image
import cv2
 
#### FOR REAL TIME COMMUNICATION BETWEEN CLIENT AND SERVER
sio = socketio.Server()
#### FLASK IS A MICRO WEB FRAMEWORK WRITTEN IN PYTHON
app = Flask(__name__)  # '__main__'
 
maxSpeed = 10
 
 
def preProcess(img):
    img = img[60:135,:,:]
    img = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
    img = cv2.GaussianBlur(img,  (3, 3), 0)
    img = cv2.resize(img, (200, 66))
    img = img/255
    return img
 
 
@sio.on('telemetry')
def telemetry(sid, data):
    speed = float(data['speed'])
    image = Image.open(BytesIO(base64.b64decode(data['image'])))
    image = np.asarray(image)
    image = preProcess(image)
    image = np.array([image])
    steering = float(model.predict(image))
    throttle = 1.0 - speed / maxSpeed
    print(f'{steering}, {throttle}, {speed}')
    sendControl(steering, throttle)
 
 
@sio.on('connect')
def connect(sid, environ):
    print('Connected')
    sendControl(0, 0)
 
 
def sendControl(steering, throttle):
    sio.emit('steer', data={
        'steering_angle': steering.__str__(),
        'throttle': throttle.__str__()
    })
 
 
if __name__ == '__main__':
    model = load_model('model.h5')
    app = socketio.Middleware(sio, app)
    ### LISTEN TO PORT 4567
    eventlet.wsgi.server(eventlet.listen(('', 4567)), app)

Setting UP
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


(12132) wsgi starting up on http://0.0.0.0:4567
(12132) accepted ('127.0.0.1', 55486)
127.0.0.1 - - [19/Mar/2024 04:33:13] "GET /socket.io/?EIO=4&transport=websocket HTTP/1.1" 200 0 110.140074
